In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import xarray as xr

In [3]:
def time_mean(ds):
    coords = ds.coords
    weights = xr.DataArray(
        ds.time_bounds.diff("time_bounds").values.astype("float64"),
        coords={"time":ds.time},
        dims=("time",)
    )
    ds_mean = ds.weighted(weights).mean("time")
    ds_mean = ds_mean.assign_coords({c:coord for (c,coord) in coords.items() if c not in ds_mean.coords})
    return ds_mean

In [8]:
toSv = 1e-6/1035.
for model in ["CM4Xp25", "CM4Xp125"]:
    wmt = xr.open_zarr(f"../../data/wmb_{model}_global_2340-2349.zarr") * toSv
    wmt_mean = time_mean(wmt.sel(exp="control"))
    wmt_mean_subset = wmt_mean[[
        'mass_tendency',
        'mass_source',
        'boundary_fluxes',
        'diffusion',
        'spurious_numerical_mixing',
        'layer_mass',
        
    ]].assign_coords(wmt_mean.coords).drop_dims(["xh", "yh"])
    wmt_mean_subset.load()
    wmt_mean_subset.to_netcdf(f"../../data/plotting_ready/wmb_{model}_global_2340-2349_time-mean.nc", mode="w")